In [1]:
cd '/Users/rabarry/Documents/Insight/NYC data/SafeEats/data/processed'

/Users/rabarry/Documents/Insight/NYC data/SafeEats/data/processed


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import seaborn as sns
import swifter
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
from sklearn import preprocessing

In [ ]:
HD_NY311 = pd.read_csv('HD_NY311.csv')

In [3]:
Yelp = pd.read_csv('Yelp_cleaned.csv')
Yelp2 = pd.read_csv('Yelp_cleaned_2.csv')

/anaconda3/envs/insight/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (9,10,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [49]:
Hoods = pd.read_csv('Hoods_zip.csv')

In [57]:
HD_NY311 = HD_NY311.drop('Unnamed: 0.1.1', axis = 1)
HD_NY311.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25647 entries, 0 to 25646
Data columns (total 32 columns):
CAMIS                                  25647 non-null int64
Second_Last_Insp_Type                  23759 non-null object
Second_Last_Insp_Date                  23759 non-null object
Second_Last_Insp_Num_CriticialFlags    23759 non-null float64
Last_Insp_Type                         25647 non-null object
Last_Insp_Date                         25647 non-null object
Last_Insp_Num_CriticialFlags           25647 non-null int64
Insp_Date_Diff                         23759 non-null float64
DBA                                    25647 non-null object
BORO                                   25647 non-null object
ZIPCODE                                25249 non-null float64
CUISINE DESCRIPTION                    25647 non-null object
Latitude                               25248 non-null float64
Longitude                              25248 non-null float64
Community Board                   

In [4]:
Yelp.info()
Yelp2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32090 entries, 0 to 32089
Data columns (total 37 columns):
Unnamed: 0                         32090 non-null int64
Unnamed: 0.1                       32090 non-null int64
yelp_alias                         32089 non-null object
yelp_categories_0_alias            31963 non-null object
yelp_categories_0_title            31963 non-null object
yelp_categories_1_alias            18968 non-null object
yelp_categories_1_title            18968 non-null object
yelp_categories_2_alias            10023 non-null object
yelp_categories_2_title            10023 non-null object
yelp_coordinates_latitude          32089 non-null object
yelp_coordinates_longitude         32089 non-null object
yelp_display_phone                 30179 non-null object
yelp_distance                      32089 non-null float64
yelp_id                            32089 non-null object
yelp_image_url                     31220 non-null object
yelp_is_closed                     32

In [50]:
Hoods.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 223 entries, 0 to 222
Data columns (total 12 columns):
Unnamed: 0                                                                  125 non-null float64
District Name                                                               223 non-null object
zip                                                                         223 non-null int64
Income diversity ratio                                                      223 non-null float64
Median household income (2018$)                                             223 non-null object
Population                                                                  223 non-null object
Population density (1,000 persons per square mile)                          223 non-null float64
Poverty rate                                                                223 non-null object
Public housing (% of rental units)                                          223 non-null object
Racial diversity index       

Merge HD, 311, and neighborhood Info

In [58]:
HD_NY311_Hoods = pd.merge(HD_NY311, Hoods, how= 'left', left_on = 'ZIPCODE', right_on = 'zip')

In [61]:
len(HD_NY311)
len(HD_NY311_Hoods)
HD_NY311_Hoods['zip'].isnull().sum()

25647

25647

398

In [62]:
HD_NY311_Hoods.head()

,CAMIS,Second_Last_Insp_Type,Second_Last_Insp_Date,Second_Last_Insp_Num_CriticialFlags,Last_Insp_Type,Last_Insp_Date,Last_Insp_Num_CriticialFlags,Insp_Date_Diff,DBA,BORO,...,zip,Income diversity ratio,Median household income (2018$),Population,"Population density (1,000 persons per square mile)",Poverty rate,Public housing (% of rental units),Racial diversity index,"Serious crime rate (per 1,000 residents)","Serious housing code violations (per 1,000 privately owned rental units)"
0,30075445,Initial,10/21/19,2.0,Re-inspection,11/4/19,1,14.0,MORRIS PARK BAKE SHOP,Bronx,...,10462.0,6.1,"$40,770","187,975",42.5,24.70%,39.80%,0.59,11.8,78.2
1,30112340,Initial,3/13/18,1.0,Initial,3/4/19,1,356.0,WENDY'S,Brooklyn,...,11225.0,5.3,"$52,050","113,212",66.3,19.10%,0.90%,0.56,12.4,84.7
2,30191841,Initial,5/16/18,1.0,Initial,6/6/19,1,386.0,DJ REYNOLDS PUB AND RESTAURANT,Manhattan,...,10019.0,7.5,"$103,930","152,455",49.6,13.80%,0.00%,0.59,90.3,8.6
3,40356018,Initial,10/30/18,1.0,Initial,11/8/19,1,374.0,RIVIERA CATERERS,Brooklyn,...,11224.0,8.2,"$36,810","122,009",35.5,24.80%,25.40%,0.63,8.4,31.6
4,40356151,Initial,8/19/17,1.0,Initial,10/6/18,2,413.0,BRUNOS ON THE BOULEVARD,Queens,...,11369.0,4.0,"$52,980","146,301",40.6,15.60%,0.00%,0.58,7.5,19.2


In [84]:
HD_NY311_Hoods.to_csv('HD_NY311_Hoods.csv')

# Add Yelp Info

Load HD_NY311_Hoods from file

In [159]:
HD_NY311_Hoods = pd.read_csv('HD_NY311_Hoods.csv')

First, remove 'None' or NaN from latitude and longitude

In [183]:
len(Yelp)
Yelp = Yelp.loc[Yelp['yelp_coordinates_longitude']!= 'None']
Yelp = Yelp.loc[Yelp['yelp_coordinates_latitude']!= 'None']
Yelp = Yelp.loc[Yelp['yelp_coordinates_latitude'].notnull()]
len(Yelp)

32090

32088

In [184]:
Yelp.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32088 entries, 0 to 32089
Data columns (total 37 columns):
Unnamed: 0                         32088 non-null int64
Unnamed: 0.1                       32088 non-null int64
yelp_alias                         32088 non-null object
yelp_categories_0_alias            31962 non-null object
yelp_categories_0_title            31962 non-null object
yelp_categories_1_alias            18967 non-null object
yelp_categories_1_title            18967 non-null object
yelp_categories_2_alias            10022 non-null object
yelp_categories_2_title            10022 non-null object
yelp_coordinates_latitude          32088 non-null object
yelp_coordinates_longitude         32088 non-null object
yelp_display_phone                 30178 non-null object
yelp_distance                      32088 non-null float64
yelp_id                            32088 non-null object
yelp_image_url                     31219 non-null object
yelp_is_closed                     32

In [260]:
Yelp['yelp_coordinates_latitude'] = Yelp['yelp_coordinates_latitude'].astype(str)
Yelp['yelp_coordinates_longitude'] = Yelp['yelp_coordinates_longitude'].astype(str)

In [261]:
Yelp['yelp_coordinates_latitude'] = [(row + '0'*(6-len(row)-1) + '1') for row in Yelp['yelp_coordinates_latitude']]

In [262]:
Yelp['yelp_coordinates_longitude'] = [(row + '0'*(7-len(row)-1) + '1') for row in Yelp['yelp_coordinates_longitude']]

In [201]:
#for row in Yelp['yelp_coordinates_latitude']:
#    if len(row)< 8:
#        row = row + '0'*(8-len(row)-1) + '1'

In [202]:
#for row in Yelp['yelp_coordinates_longitude']:
#    if len(row)< 9:
#        row = row + '0'*(9-len(row))

In [263]:
Yelp['yelp_coordinates_latitude'] = pd.to_numeric(Yelp['yelp_coordinates_latitude'])
Yelp['yelp_coordinates_longitude'] = pd.to_numeric(Yelp['yelp_coordinates_longitude'])

In [264]:
Yelp['yelp_coordinates_latitude'] = Yelp['yelp_coordinates_latitude'].round(3)
Yelp['yelp_coordinates_longitude'] = Yelp['yelp_coordinates_longitude'].round(3)
Yelp['location'] = Yelp['yelp_coordinates_latitude'].astype(str) + ', ' + Yelp['yelp_coordinates_longitude'].astype(str)

In [265]:
Yelp.iloc[1]

Unnamed: 0                                                                         1
Unnamed: 0.1                                                                       2
yelp_alias                                              kingsbridge-donut-shop-bronx
yelp_categories_0_alias                                                       donuts
yelp_categories_0_title                                                       Donuts
yelp_categories_1_alias                                             breakfast_brunch
yelp_categories_1_title                                           Breakfast & Brunch
yelp_categories_2_alias                                                          NaN
yelp_categories_2_title                                                          NaN
yelp_coordinates_latitude                                                     40.881
yelp_coordinates_longitude                                                   -73.906
yelp_display_phone                                               

In [266]:
HD_NY311_Hoods['Latitude'] = HD_NY311_Hoods['Latitude'].round(3)
HD_NY311_Hoods['Longitude'] = HD_NY311_Hoods['Longitude'].round(3)
HD_NY311_Hoods['location'] = HD_NY311_Hoods['Latitude'].astype(str) + ', ' + HD_NY311_Hoods['Longitude'].astype(str)

In [267]:
Yelp['yelp_name'] = Yelp['yelp_name'].swifter.apply(lambda x: x.lower())
HD_NY311_Hoods['DBA'] = HD_NY311_Hoods['DBA'].swifter.apply(lambda x: x.lower())
Yelp['yelp_name']
HD_NY311_Hoods['DBA']

0              brooklyn sweet spot
1           kingsbridge donut shop
2                    casa mexicana
3                   the palm court
4           dominique ansel bakery
                   ...            
32085      brooklyn best loctician
32086    wonder seafood restaurant
32087              tenement museum
32088                studio b hair
32089        infinity caterers inc
Name: yelp_name, Length: 32088, dtype: object

0                 morris park bake shop
1                               wendy's
2        dj reynolds pub and restaurant
3                      riviera caterers
4               brunos on the boulevard
                      ...              
25642                 under the volcano
25643                  lic landing cafe
25644     tandoori curry & indo chinese
25645                     zhu's kitchen
25646                          baya bar
Name: DBA, Length: 25647, dtype: object

In [268]:
HD_NY311_Hoods = HD_NY311_Hoods.swifter.apply(lambda x: x.str.strip() if x.dtype =='object' else x)
Yelp['location'] = Yelp['location'].swifter.apply(lambda x: x.strip())

In [269]:
HD_NY311_Hoods_Yelp_TEST = pd.merge(HD_NY311_Hoods, Yelp, how= 'left', left_on = ['location', 'DBA'], right_on = ['location', 'yelp_name'])

In [271]:
len(HD_NY311_Hoods)

25647

In [270]:
len(HD_NY311_Hoods_Yelp_TEST)
#HD_NY311_Hoods_Yelp_TEST.head()
HD_NY311_Hoods_Yelp_TEST['yelp_review_count'].unique()
HD_NY311_Hoods_Yelp_TEST.info()

25674

,Unnamed: 0_x,CAMIS,Second_Last_Insp_Type,Second_Last_Insp_Date,Second_Last_Insp_Num_CriticialFlags,Last_Insp_Type,Last_Insp_Date,Last_Insp_Num_CriticialFlags,Insp_Date_Diff,DBA,...,yelp_phone,yelp_price,yelp_rating,yelp_review_count,yelp_transactions_0,yelp_transactions_1,yelp_transactions_2,yelp_url,yelp_result_num,new_latitude
0,0,30075445,Initial,10/21/19,2.0,Re-inspection,11/4/19,1,14.0,morris park bake shop,...,1.718892e+10,$$,4.0,43.0,NaN,NaN,NaN,https://www.yelp.com/biz/morris-park-bake-shop...,0.0,40.848391
1,1,30112340,Initial,3/13/18,1.0,Initial,3/4/19,1,356.0,wendy's,...,1.718288e+10,$,2.0,39.0,NaN,NaN,NaN,https://www.yelp.com/biz/wendys-brooklyn-4?adj...,0.0,40.6629521
2,2,30191841,Initial,5/16/18,1.0,Initial,6/6/19,1,386.0,dj reynolds pub and restaurant,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,40356018,Initial,10/30/18,1.0,Initial,11/8/19,1,374.0,riviera caterers,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,40356151,Initial,8/19/17,1.0,Initial,10/6/18,2,413.0,brunos on the boulevard,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 25674 entries, 0 to 25673
Data columns (total 83 columns):
Unnamed: 0_x                                                                25674 non-null int64
CAMIS                                                                       25674 non-null int64
Second_Last_Insp_Type                                                       23784 non-null object
Second_Last_Insp_Date                                                       23784 non-null object
Second_Last_Insp_Num_CriticialFlags                                         23784 non-null float64
Last_Insp_Type                                                              25674 non-null object
Last_Insp_Date                                                              25674 non-null object
Last_Insp_Num_CriticialFlags                                                25674 non-null int64
Insp_Date_Diff                                                              23784 non-null float64
DBA       

In [224]:
Yelp.location[100]
HD_NY311_Hoods.location[100]

'40.76381, -73.97518'

'40.70534, -74.00869'